In [2]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
import locale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

2023-11-14 23:01:10.769155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
def create_cnn(width, height, depth, filters=(16, 32, 64)):
    model = Sequential()
    model.add(Conv2D(64,kernel_size=(3,3), activation = 'relu', input_shape = (320,320,9), kernel_regularizer = 'l2'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(Conv2D(32,kernel_size=(3,3),activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(Conv2D(16,kernel_size=(3,3),activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(8,activation = 'relu'))
    model.add(Dense(4,activation = 'relu'))
    return model

In [19]:
def create_mlp(dim):
	# define our MLP network
  model = Sequential()
  model.add(Dense(128, activation = 'relu', input_shape=(dim,),kernel_regularizer = 'l2'))
  model.add(Dense(64, activation = 'relu'))
  model.add(Dense(16, activation = 'relu'))
  return model

In [20]:
gasdata = pd.read_csv("/Users/wzhang/Documents/data5.csv")
data_dir = '/Users/wzhang/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        img = mpimg.imread('/Users/wzhang/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)


1000


In [21]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX)
print(trainy)


          MQ2       MQ7
623  0.821114  0.205279
750  0.929619  0.800587
996  0.953079  0.771261
349  0.900293  0.785924
261  0.882698  0.715543
..        ...       ...
630  0.810850  0.000000
69   0.891496  0.673021
942  0.939883  0.756598
9    0.926686  0.740469
436  0.787390  0.551320

[800 rows x 2 columns]
623    1
750    1
996    1
349    0
261    0
      ..
630    1
69     0
942    1
9      0
436    0
Name: Gas, Length: 800, dtype: int64


In [22]:
mlp = create_mlp(trainAttrX.shape[1])
cnn = create_cnn(320, 320, 9)
combinedInput = concatenate([mlp.output, cnn.output])
print(mlp.output)
print(cnn.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), name='dense_11/Relu:0', description="created by layer 'dense_11'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='dense_13/Relu:0', description="created by layer 'dense_13'")


In [23]:
x = Dense(8, activation="relu")(combinedInput)
x = Dense(1, activation = "sigmoid")(x)

In [24]:
model = Model(inputs=[mlp.input, cnn.input],outputs = x)


In [25]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), loss=tf.losses.BinaryCrossentropy(),metrics = ['accuracy']) 
print(model.summary)


In [26]:
hist = model.fit(x=[trainAttrX, trainImagesX], y=trainy,validation_data=([testAttrX, testImagesX], testy),epochs=10)


Epoch 1/10
25/25 [==============================] - 246s 9s/step - loss: 0.8832 - accuracy: 0.5225 - val_loss: 0.8753 - val_accuracy: 0.5900
Epoch 2/10
 9/25 [=========>....................] - ETA: 1:58 - loss: 0.8723 - accuracy: 0.6354

In [ ]:
print(hist.history)


In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color = 'teal', label = 'loss')
plt.plot(hist.history['val_loss'], color = 'orange', label = 'val_loss')
fig.suptitle('Loss', fontsize = 20)
plt.legend(loc = "upper left")
plt.show()

In [ ]:
fig2 = plt.figure()
plt.plot(hist.history['accuracy'], color = 'teal', label = 'accuracy')
plt.plot(hist.history['val_accuracy'], color = 'orange', label = 'accuracy')
fig2.suptitle('Accuracy', fontsize = 20)
plt.legend(loc = "upper left")
plt.show()

In [ ]:
accuracy = model.evaluate([testAttrX, testImagesX],testy)
print(accuracy)
performance = model.predict([testAttrX,testImagesX])
actual = []
for value in performance: 
    if(value>=0.5):
         actual.append(1)
    else:
        actual.append(0)
precision = precision_score(testy, actual)
print('Precision: %f' % precision)
recall = recall_score(testy, actual)
print('Recall: %f' % recall)
f1 = f1_score(testy, actual)
print('F1 score: %f' % f1)

In [ ]:
cm = confusion_matrix(testy, actual)
sns.heatmap(cm, 
    annot=True,
    fmt='g', 
    xticklabels=['Gas','No Gas'],
    yticklabels=['Gas','No Gas'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()